### Libs

In [1]:
# Importações gerais
import os
import re
import unicodedata

# Manipulação de dados
import pandas as pd

### Files

In [2]:
# Caminho da pasta onde estão os arquivos
folder_path = '/Users/luisfaria/Desktop/sEngineer/dash/db/campaign'
files = os.listdir(folder_path)

# Inicializar os DataFrames como None
df_agd = None
df_leads = None

# Caminho da pasta onde estão os arquivos
files = os.listdir(folder_path)

# Verificar os arquivos e carregar os que correspondem
for file_name in files:
    file_path = os.path.join(folder_path, file_name)

    # Verificar se é um arquivo Excel
    if file_name.endswith('.xlsx'):
        if 'agd' in file_name.lower():
            df_agd = pd.read_excel(file_path)
            print(f"Arquivo de agendamentos carregado: {file_name}")
        elif 'lead' in file_name.lower():
            df_leads = pd.read_excel(file_path)
            print(f"Arquivo de leads carregado: {file_name}")
# Verificar se todos os DataFrames foram carregados
if df_agd is None:
    print("Nenhum arquivo de agendamentos encontrado.")
if df_leads is None:
    print("Nenhum arquivo de leads encontrado.")

Arquivo de agendamentos carregado: agd.xlsx
Arquivo de leads carregado: lead.xlsx


In [ ]:
df_leads.columns

In [ ]:
df_agd.columns

### Cleaning Apt

In [ ]:
# Filter appointments based on multiple conditions
df_agd_comparecimentos_da_campanha = df_agd[
    # Condition 1: Procedimento is either "AVALIAÇÃO INJETÁVEIS E INVASIVOS" or "AVALIAÇÃO ESTÉTICA"
    ((df_agd['Procedimento'] == "AVALIAÇÃO INJETÁVEIS E INVASIVOS") | (df_agd['Procedimento'] == "AVALIAÇÃO ESTÉTICA")) &
    # Condition 2: Status is "Atendido"
    (df_agd['Status'] == "Atendido") &
    # Condition 3: Fonte de cadastro is "Recepcionista" or Prestador is "Julia Bigliazzi Amorim Nogueira"
    ((df_agd['Grupo da primeira atendente'] == "Recepcionista") | (df_agd['Nome da primeira atendente'] == "Julia Bigliazzi Amorim Nogueira"))
]

df_agd_comparecimentos_da_campanha

### Task

In [ ]:
"""
Now I need to:
create a new column at df_agd_comparecimentos_da_campanha:
- 'eh_lead' to check either if email or Telefone matches any of the fields from df_leads['Telefone do lead'] (just true or false)
- 'fonte_cliente' to check if df_agd_comparecimentos_da_campanha['Fonte de cadastro do cliente'] is "Indicação" or "Fachada"?
- 'eh_reagendado' to check if df_agd_comparecimentos_da_campanha['Telefone'] exists with any row of df_agd['Telefone'] where df_agd['Status'] == 'Reagendado'
"""

In [ ]:
# 1. Create 'eh_lead' column - checks if email or phone matches any lead's phone
# First, ensure both DataFrames have standardized phone formats for comparison
def standardize_phone(phone_str):
    if pd.isna(phone_str):
        return ""
    # Remove non-digit characters
    return re.sub(r'\D', '', str(phone_str))

# Apply standardization to relevant columns
if 'Telefone do lead' in df_leads.columns:
    df_leads['phone_standardized'] = df_leads['Telefone do lead'].apply(standardize_phone)
df_agd_comparecimentos_da_campanha['phone_standardized'] = df_agd_comparecimentos_da_campanha['Telefone'].apply(standardize_phone)

# Create the 'eh_lead' column by checking for matches
lead_phones = set(df_leads['phone_standardized'].dropna())
df_agd_comparecimentos_da_campanha['eh_lead'] = False

# Check for phone matches
df_agd_comparecimentos_da_campanha.loc[
    df_agd_comparecimentos_da_campanha['phone_standardized'].isin(lead_phones), 'eh_lead'] = True

# Check for email matches (if email columns exist)
if 'Email' in df_agd_comparecimentos_da_campanha.columns and 'Email do lead' in df_leads.columns:
    lead_emails = set(df_leads['Email do lead'].dropna())
    df_agd_comparecimentos_da_campanha.loc[
        df_agd_comparecimentos_da_campanha['Email'].isin(lead_emails), 'eh_lead'] = True

# 2. Create 'fonte_cliente' column - checking if source is "Indicação" or "Fachada"
df_agd_comparecimentos_da_campanha['fonte_cliente'] = df_agd_comparecimentos_da_campanha['Fonte de cadastro do cliente'].isin(["Indicação", "Fachada"])

# 3. Create 'eh_reagendado' column - check if phone exists in rescheduled appointments
# Get all phones from rescheduled appointments
reagendado_phones = set(standardize_phone(phone) for phone in 
                        df_agd.loc[df_agd['Status'] == 'Reagendado', 'Telefone'])

# Check if current appointment phones are in the rescheduled set
df_agd_comparecimentos_da_campanha['eh_reagendado'] = df_agd_comparecimentos_da_campanha['phone_standardized'].isin(reagendado_phones)

# Clean up the temporary standardized phone column if not needed for further analysis
# df_agd_comparecimentos_da_campanha.drop('phone_standardized', axis=1, inplace=True)

# Display the first few rows with the new columns
print(f"Total rows with new columns: {len(df_agd_comparecimentos_da_campanha)}")
df_agd_comparecimentos_da_campanha[['Nome cliente', 'eh_lead', 'fonte_cliente', 'eh_reagendado']].head()

In [ ]:
df_agd_comparecimentos_da_campanha.columns

In [5]:
coluns_to_be_kept = ['ID agendamento', 'ID cliente', 'Nome cliente', 'Fonte de cadastro do cliente',
       'Unidade do agendamento', 'Procedimento', 'Data', 'Status', 'Nome da primeira atendente',
       'Última data de alteração do status',
       'eh_lead', 'fonte_cliente', 'eh_reagendado']

df_agd_comparecimentos_da_campanha_filtered = df_agd_comparecimentos_da_campanha[coluns_to_be_kept]

df_agd_comparecimentos_da_campanha_filtered.to_excel('r_agendamentos_compareceu_ganhou_2025_fev.xlsx', index=False)

In [ ]:
# Groupby df_agd_comparecimentos_da_campanha_filtered('Nome da primeira atendente').agg 

In [46]:
# Now create a groupby aggregation by 'Nome da primeira atendente'
atendente_summary = df_agd_comparecimentos_da_campanha_filtered.groupby('Nome da primeira atendente').agg({
    'ID agendamento': 'count',  # Count of appointments
    'eh_lead': 'sum',           # Count of appointments that are leads (True counts as 1)
    'fonte_cliente': 'sum',     # Count of appointments from "Indicação" or "Fachada"
    'eh_reagendado': 'sum'      # Count of rescheduled appointments
}).reset_index()

# Rename columns for clarity
atendente_summary.rename(columns={
    'ID agendamento': 'Total_Agendamentos',
    'eh_lead': 'Total_Leads',
    'fonte_cliente': 'Total_Indicacao_Fachada',
    'eh_reagendado': 'Total_Reagendados'
}, inplace=True)

# Calculate percentages
atendente_summary['Pct_Leads'] = (atendente_summary['Total_Leads'] / atendente_summary['Total_Agendamentos'] * 100).round(1)
atendente_summary['Pct_Indicacao_Fachada'] = (atendente_summary['Total_Indicacao_Fachada'] / atendente_summary['Total_Agendamentos'] * 100).round(1)
atendente_summary['Pct_Reagendados'] = (atendente_summary['Total_Reagendados'] / atendente_summary['Total_Agendamentos'] * 100).round(1)

# Sort by total appointments (descending)
atendente_summary = atendente_summary.sort_values('Total_Agendamentos', ascending=False)

# Display the summary
print(f"Análise por Atendente - Total de {len(atendente_summary)} atendentes")
atendente_summary

# Save the summary to Excel
atendente_summary.to_excel('r_resumo_atendentes_compareceu_ganhou_2025_fev.xlsx', index=False)

# Original export of the filtered data
df_agd_comparecimentos_da_campanha_filtered.to_excel('r_agendamentos_compareceu_ganhou_2025_fev.xlsx', index=False)

Análise por Atendente - Total de 34 atendentes


In [11]:
df_agd_comparecimentos_da_campanha_filtered
"""
Pedidos novos:
coluna nova 1 = é cliente antigo? Considerando que clientes novos são apenas os criados em 2025
coluna nova 2 = se cliente teve agendamento 6 meses atrás sim ou não
"""

df_agd_comparecimentos_da_campanha_filtered['cliente_antigo'] = df_agd_comparecimentos_da_campanha_filtered['ID cliente'] <= 1009616


/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_78124/989244285.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agd_comparecimentos_da_campanha_filtered['cliente_antigo'] = df_agd_comparecimentos_da_campanha_filtered['ID cliente'] <= 1009616


,ID agendamento,ID cliente,Nome cliente,Fonte de cadastro do cliente,Unidade do agendamento,Procedimento,Data,Status,Nome da primeira atendente,Última data de alteração do status,eh_lead,fonte_cliente,eh_reagendado,cliente_antigo
1427,2827534,1011048,Regina De Oliveira Damasceno,CRM BÔNUS,LAPA,AVALIAÇÃO INJETÁVEIS E INVASIVOS,01/02/2025,Atendido,Maria Idielma da Silva Fabrício,01/02/2025 10:38,False,False,False,False
1558,2829029,1012802,João Guilherme Siqueira,Facebook Leads,TATUAPÉ,AVALIAÇÃO INJETÁVEIS E INVASIVOS,01/02/2025,Atendido,Letícia Moreira Valentim,01/02/2025 13:44,False,False,False,False
1756,2831754,1013389,Laiana Jager Matthews,Google Pesquisa,COPACABANA,AVALIAÇÃO INJETÁVEIS E INVASIVOS,15/02/2025,Atendido,Renally Carla da Silva Moura,15/02/2025 13:09,False,False,False,False
1880,2834081,1013726,Beatriz Boaventura,Busca Orgânica,ITAIM,AVALIAÇÃO ESTÉTICA,04/02/2025,Atendido,Vanessa Alcântara de Sousa,04/02/2025 18:37,False,False,False,False
1929,2834683,1010994,Rafaela Rapini Santos,Instagram,ALPHAVILLE,AVALIAÇÃO ESTÉTICA,08/02/2025,Atendido,Larissa Rodrigues da Silva,06/03/2025 13:01,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38948,2910006,1023655,Tatiane Xavier da Silva,Indique e Multiplique,JARDINS,AVALIAÇÃO INJETÁVEIS E INVASIVOS,26/02/2025,Atendido,Marta Maria Palma,06/03/2025 10:55,True,False,True,False
38949,2910026,1023655,Tatiane Xavier da Silva,Indique e Multiplique,JARDINS,AVALIAÇÃO INJETÁVEIS E INVASIVOS,25/02/2025,Atendido,Marta Maria Palma,06/03/2025 11:01,True,False,True,False
39103,2911318,1019169,Richard São Felippe,Google Pesquisa,TUCURUVI,AVALIAÇÃO INJETÁVEIS E INVASIVOS,19/02/2025,Atendido,Camylli Victoria Lonis Silva,10/03/2025 09:31,False,False,True,False
39208,2913143,1021249,Silmara Aparecida Ribeiro Pilan,Acesso Direto ao Site,MOOCA,AVALIAÇÃO INJETÁVEIS E INVASIVOS,18/02/2025,Atendido,Julia Bigliazzi Amorim Nogueira,07/03/2025 16:31,True,False,True,False


### 6months ago

In [13]:
folder_path = '/Users/luisfaria/Desktop/sEngineer/dash/db/campaign/agd_6m'
files = os.listdir(folder_path)

# Filter out files that are not Excel files
excel_files = [file for file in files if file.endswith('.xlsx')]

# Create an empty list to store DataFrames
dfs = []

# Read each Excel file and append to the list
for file in excel_files:
    file_path = os.path.join(folder_path, file)
    print(f"Reading {file}")  # Optional: to track progress
    df = pd.read_excel(file_path)
    dfs.append(df)

# Concatenate all DataFrames into one
df_agd_6m = pd.concat(dfs, ignore_index=True)

Reading agd_2024_09.xlsx
Reading agd_2024_12.xlsx
Reading agd_2024_08.xlsx
Reading agd_2024_07.xlsx
Reading agd_2024_11.xlsx
Reading agd_2024_10.xlsx


In [ ]:
# Using OR operator (|) instead of AND (&)
aval = df_agd_6m.loc[(df_agd_6m['Procedimento'] == 'AVALIAÇÃO INJETÁVEIS E INVASIVOS') | 
                            (df_agd_6m['Procedimento'] == 'AVALIAÇÃO ESTÉTICA')]

In [36]:
df_agd_comparecimentos_da_campanha_filtered['teve_agendamento_6m'] = df_agd_comparecimentos_da_campanha_filtered['ID cliente'].isin(avals['ID cliente'])

/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_78124/1741957206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agd_comparecimentos_da_campanha_filtered['teve_agendamento_6m'] = df_agd_comparecimentos_da_campanha_filtered['ID cliente'].isin(avals['ID cliente'])


### Download final

In [45]:
coluns_to_be_kept = ['ID agendamento', 'ID cliente', 'Nome cliente', 'Fonte de cadastro do cliente',
       'Unidade do agendamento', 'Procedimento', 'Data', 'Status', 'Nome da primeira atendente',
       'Última data de alteração do status',
       'eh_lead', 'fonte_cliente', 'eh_reagendado', 'cliente_antigo', 'teve_agendamento_6m']

df_agd_comparecimentos_da_campanha_filtered = df_agd_comparecimentos_da_campanha_filtered[coluns_to_be_kept]

df_agd_comparecimentos_da_campanha_filtered.to_excel('r_agendamentos_compareceu_ganhou_2025_fev.xlsx', index=False)